In [1]:
import json
import os
import torch
from torchvision import transforms
from tqdm import tqdm

from time import time as t

from bindsnetExt.Datasets import *
from bindsnetExt.Network import *
from bindsnetExt.Learning import *

from bindsnet.encoding import PoissonEncoder
from bindsnet.network.monitors import Monitor
from bindsnet.utils import get_square_weights, get_square_assignments
from bindsnet.evaluation import all_activity, proportion_weighting, assign_labels

from sklearn.model_selection import StratifiedKFold
from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import pickle

import matplotlib.pyplot as plt

In [22]:
def runExperiment(params, lrnArgs, p_type, save_data = True):
        
         
         
    if p_type == "ppx":
        LR = ppxPostPre
    elif p_type == "nc":
        LR = ncPostPre
    else:
        LR = PostPre

    
    time = int(params["time"])
    intensity = params["intensity"]
    n_neurons = int(params["n_neurons"])
    dt = params['dt']
    
    
    progress_interval = 3
    update_interval = int(params["update_interval"])
    seed = 0
    n_epochs = int(params["n_epochs"])
    n_workers = 0

    gpu = params["gpu"]
    
    excArgs = {'rest':params["rest"],
               'reset':params["reset"],
               'thresh':params['thresh'], 
               'refrac':params['refrac'], 
               'tc_decay':params['tc_decay'], 
               'tc_trace':params["tc_trace"]}
    
    lrn_keys = [key for key in lrnArgs]
    
    nu = (lrnArgs[lrn_keys[1]], lrnArgs[lrn_keys[0]])

    # Sets up Gpu use
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if gpu and torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    else:
        torch.manual_seed(seed)
        device = "cpu"
        if gpu:
            gpu = False
    
    np.random.seed(seed)

    torch.set_num_threads(os.cpu_count() - 1)
    print("Running on Device = ", device)
    
    
    X, Y = load_digits(n_class=10, return_X_y=True, as_frame=False)
    
    n_inpt = X.shape[-1]

    
        
    skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = seed)
    
    accs = []
    prop = []
    logreg = []
    
    fold_num = 0

    for train_idx, test_idx in skf.split(X,Y):
        
        # Build network.
        network = DiehlAndCook(
            LearningRule = LR,
            n_inpt=n_inpt,
            nu = nu,
            wmin = 0.,
            wmax = 1.,
            inh=params["inh"],
            inpt_shape=(1, int(np.sqrt(n_inpt)), int(np.sqrt(n_inpt))),
            n_neurons = n_neurons,
            theta_plus = 0.07,
            dt = dt,
            norm = params["norm"],
            tc_theta_decay = 7e6,
            lrnArgs = lrnArgs,
            excArgs = excArgs,   
        )

        LM = LogisticRegression(C=params["C"])


        # Directs network to GPU
        if gpu:
            network.to("cuda")
        
        
        
        x_train, y_train = X[train_idx], Y[train_idx]
        x_test, y_test = X[test_idx], Y[test_idx]
        

        train_dataset = bnDataset(
                                PoissonEncoder(time=time, dt=dt),
                                None,
                                transform=transforms.Compose(
                                    [transforms.ToTensor(), transforms.Lambda(lambda x: x * intensity)]
                                ),

                                  subset_x = x_train, 
                                  subset_y = y_train, 
                                  shape = (int(np.sqrt(n_inpt)), int(np.sqrt(n_inpt))),
                                 )

        test_dataset = bnDataset(
                                PoissonEncoder(time=time, dt=dt),
                                None,
                                transform=transforms.Compose(
                                    [transforms.ToTensor(), transforms.Lambda(lambda x: x * intensity)]
                                ),

                                  subset_x = x_test, 
                                  subset_y = y_test, 
                                  shape = (int(np.sqrt(n_inpt)), int(np.sqrt(n_inpt))),
                                 )


        n_train = len(train_dataset) // 10
        print("n_train: ", n_train)

        # Record spikes during the simulation.
        spike_record = torch.zeros((update_interval, int(time / dt), n_neurons), device=device)

        # Neuron assignments and spike proportions.
        n_classes = len(np.unique(Y))
        assignments = -torch.ones(n_neurons, device=device)
        proportions = torch.zeros((n_neurons, n_classes), device=device)
        rates = torch.zeros((n_neurons, n_classes), device=device)


        # Set up monitors for spikes and voltages
        spikes = {}
        for layer in set(network.layers):
            spikes[layer] = Monitor(
                network.layers[layer], state_vars=["s"], time=int(time / dt)
            )
            network.add_monitor(spikes[layer], name="%s_spikes" % layer)

        # Train the network.
        print("\nBegin training.\n")
#         print("network layers: ", network.layers)
        start = t()
        for epoch in range(n_epochs):
            labels = []

            if epoch % progress_interval == 0:
                #print("Progress: %d / %d (%.4f seconds)" % (epoch, n_epochs, t() - start))
                start = t()

            # Create a dataloader to iterate and batch data
            dataloader = torch.utils.data.DataLoader(
                train_dataset, batch_size=1, shuffle=True, num_workers=n_workers, pin_memory=gpu
            )

            for step, batch in enumerate(dataloader):
                if step > n_train:
                    break
                # Get next input sample.
                inputs = {"X": batch["encoded_image"].view(int(time / dt), 1, 1, int(np.sqrt(n_inpt)), int(np.sqrt(n_inpt)))}
                if gpu:
                    inputs = {k: v.cuda() for k, v in inputs.items()}
                    
                network.train(False)
                # Run the network on the input.
                network.run(inputs=inputs, time=time, input_time_dim=1)
                
                # Add to spikes recording.
                spike_record[step % update_interval] = spikes["Y"].get("s").squeeze()

                # Convert the array of labels into a tensor
                label_tensor = torch.tensor(labels, device=device)

                # Get network predictions.
                all_activity_pred = all_activity(
                    spikes=spike_record, assignments=assignments, n_labels=n_classes
                )

                proportion_pred = proportion_weighting(
                    spikes=spike_record,
                    assignments=assignments,
                    proportions=proportions,
                    n_labels=n_classes,
                )


                # Assign labels to excitatory layer neurons.
                assignments, proportions, rates = assign_labels(
                    spikes=spike_record,
                    labels=label_tensor,
                    n_labels=n_classes,
                    rates=rates,
                )

                labels = []

                labels.append(batch["label"])
                
                y_act = all_activity_pred.cpu().numpy()
                y_prop = proportion_pred.cpu().numpy()
                
                if (y_act == batch["label"]):
                    reward = 1
                else:
                    reward = -1
                    
                network.reset_state_variables()  # Reset state variables.
                
                network.train(True)
                network.run(inputs=inputs, time=time, input_time_dim=1, reward=reward)
                
                network.reset_state_variables()  # Reset state variables.


        n_test = len(test_dataset) // 10

        # Sequence of accuracy estimates.
        accuracy = {"all": 0, "proportion": 0}

        # Record spikes during the simulation.
        spike_record = torch.zeros((1, int(time / dt), n_neurons), device=device)

        # Train the network.
        print("\nBegin testing\n")
        network.train(mode=False)
        start = t()
        
        y_act = []
        y_prop = []

        #pbar = tqdm(total=n_test)
        for step, batch in enumerate(test_dataset):
            if step >= n_test:
                break
            # Get next input sample.
            inputs = {"X": batch["encoded_image"].view(int(time / dt), 1, 1, int(np.sqrt(n_inpt)), int(np.sqrt(n_inpt)))}
            if gpu:
                inputs = {k: v.cuda() for k, v in inputs.items()}

            # Run the network on the input.
            network.run(inputs=inputs, time=time, input_time_dim=1)

            # Add to spikes recording.
            spike_record[0] = spikes["Y"].get("s").squeeze()

            # Convert the array of labels into a tensor
            label_tensor = torch.tensor(batch["label"], device=device)

            # Get network predictions.
            all_activity_pred = all_activity(
                spikes=spike_record, assignments=assignments, n_labels=n_classes
            )

            proportion_pred = proportion_weighting(
                spikes=spike_record,
                assignments=assignments,
                proportions=proportions,
                n_labels=n_classes,
            )
            

            # Compute network accuracy according to available classification strategies.
            accuracy["all"] += float(torch.sum(label_tensor.long() == all_activity_pred).item())
            accuracy["proportion"] += float(
                torch.sum(label_tensor.long() == proportion_pred).item()
            )
            
            y_act.append(all_activity_pred.cpu().numpy())
            y_prop.append(proportion_pred.cpu().numpy())


            network.reset_state_variables()  # Reset state variables.
        
        if save_data:
        
            with open(f"./expResults/digits_labels/fold_{fold_num}_activity_pred_{p_type}.pkl", 'wb') as f:
                pickle.dump(np.asarray(y_act), f)

            with open(f"./expResults/digits_labels/fold_{fold_num}_proportion_pred_{p_type}.pkl", 'wb') as f:
                pickle.dump(np.asarray(y_prop), f)


            weights = network.connections[('X', 'Y')].w.cpu().numpy()
            
            with open(f"./expResults/digits_weights/fold_{fold_num}_weights_{p_type}.pkl", 'wb') as f:
                pickle.dump(weights, f)
                
            with open(f"./expResults/digits_assignments/fold_{fold_num}_assignments_{p_type}.pkl", 'wb') as f:
                pickle.dump(assignments.cpu().numpy(), f)
                
            with open(f"./expResults/digits_indexing/fold_{fold_num}_train_idx_{p_type}.pkl", 'wb') as f:
                pickle.dump(train_idx, f)
        
            
        fold_num+=1
        
        print("Testing complete.\n")
        
        acc = accuracy["all"]/n_test
        print(acc)
        
        accs.append(acc)
        prop.append(accuracy["proportion"]/n_test)
        
        del train_dataset
        del test_dataset
        del network
        
        torch.cuda.empty_cache()
        
        
        
    print(f"voting accuracy: {np.mean(accs)}")
    print(f"voting proportion accuracy: {np.mean(prop)}")
        
    return (accs, prop)

In [23]:
with open("./Parameters/digits_memristors.json", 'r', encoding='utf-8') as f:
        parameters = json.load(f)

In [24]:
parameters

{'ppx': [{'time': 15,
   'dt': 1,
   'n_neurons': 160,
   'intensity': 82.69,
   'update_interval': 1,
   'inh': 15.2,
   'rest': -65,
   'reset': -65,
   'thresh': -28.83,
   'refrac': 5.78,
   'tc_decay': 120.29,
   'tc_trace': 10,
   'norm': 62.14,
   'n_epochs': 3,
   'gpu': True,
   'C': 1000.0},
  {'alpha_pos': 0.316,
   'alpha_neg': 0.011,
   'beta_pre': 2.213,
   'beta_post': -5.969,
   'gamma_pre': 0.032,
   'gamma_post': 0.146}],
 'ppx_rstdp': [{'time': 15,
   'dt': 1,
   'n_neurons': 160,
   'intensity': 82.69,
   'update_interval': 1,
   'inh': 15.2,
   'rest': -65,
   'reset': -65,
   'thresh': -28.83,
   'refrac': 5.78,
   'tc_decay': 120.29,
   'tc_trace': 10,
   'norm': 62.14,
   'n_epochs': 3,
   'gpu': True,
   'C': 1000.0},
  {'alpha_pos': 0.316,
   'alpha_neg': 0.011,
   'beta_pre': 2.213,
   'beta_post': -5.969,
   'gamma_pre': 0.032,
   'gamma_post': 0.146}],
 'nc': [{'time': 300,
   'dt': 1,
   'n_neurons': 1600,
   'intensity': 3.98,
   'update_interval': 300,
 

In [25]:
key = 'ppx_rstdp'

In [26]:
exp = runExperiment(parameters[key][0], parameters[key][1], key, save_data = True)

Running on Device =  cpu
n_train:  143

Begin training.


Begin testing

Testing complete.

0.027777777777777776
n_train:  143

Begin training.




KeyboardInterrupt

